In [2]:
import sys
import cv2
import numpy as np

In [4]:
def gray_canny(img_original):
    """
        Run grayscale and canny edge detector to simplify an image
    """
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray,100,100,apertureSize = 3)

    return canny

def draw_corners(img_original):
    """
        Draws corners over and returns the image
    """
    img = gray_canny(img_original)

    img = np.float32(img)
    dst = cv2.cornerHarris(img,2,3,0.04)

    dst = cv2.dilate(dst,None)

    # Threshold for an optimal value, it may vary depending on the image.
    img[dst>0.01*dst.max()]=[0,0,255]

    return img

def draw_contours(img_original):
    """
        Contours
    """
    img = gray_canny(img_original)
    contours, hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    cv2.drawContours(img_original,contours,-1,(0,255,0),3)

    return img_original

def draw_lines(img_original):
    """
        Draws Hough lines over and returns the image
    """
    img = gray_canny(img_original)
    minLineLength = 100
    maxLineGap = 10
    lines = cv2.HoughLinesP(img,1,np.pi/180,200,minLineLength,maxLineGap)
    for x1,y1,x2,y2 in lines[0]:
        cv2.line(img_original,(x1,y1),(x2,y2),(0,255,0),2)

    return img_original

def draw_broad_lines(img_original):
    """
        Aggregates and extends the lines drawn in draw_lines()
        to the edges of the screen
    """
    shape = img_original.shape
    height = shape[0]
    width = shape[1]

    img = gray_canny(img_original)
    minLineLength = 100
    maxLineGap = 10
    lines = cv2.HoughLinesP(img,1,np.pi/180,200,minLineLength,maxLineGap)[0].tolist()

    # discard lines that are neither horizontal nor vertical
    lines_horizontal = [line for line in lines if line[1] == line[3]]
    lines_vertical = [line for line in lines if line[0] == line[2]]

    # extend horizontal and vertical lines to the edges of the screen
    lines_horizontal_ext = [[0, line[1], width, line[3]] for line in lines_horizontal]
    lines_vertical_ext = [[line[0], 0, line[2], height] for line in lines_vertical]

    # convert to strings and deduplicate
    lines_horizontal_ext_unique = [[int(x) for x in line_string.split(",")] for line_string in list(set([",".join([str(x) for x in line]) for line in lines_horizontal_ext]))]
    lines_vertical_ext_unique = [[int(x) for x in line_string.split(",")] for line_string in list(set([",".join([str(x) for x in line]) for line in lines_vertical_ext]))]

    lines_broad = lines_horizontal_ext_unique + lines_vertical_ext_unique

    for x1,y1,x2,y2 in lines_broad:
        cv2.line(img_original,(x1,y1),(x2,y2),(0,255,0),2)

    return img_original